In [1]:
!pip install git+https://github.com/openai/CLIP.git torch pillow

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-gkknv3_s
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-gkknv3_s
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=d3c4620c2ff9b141e227b8f213bb4bac196012d089f61adbed8e5d881788c295
  Stored in directory: /tmp/pip-ephem-wheel-cache-468hycsk/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [2]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 29.2 MB/s eta 0:00:00


In [3]:
import clip
import torch
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [4]:
from google.colab import userdata

In [5]:
username = userdata.get('MONGO_USERNAME')
password = userdata.get('MONGO_PASSWORD')
cluster = userdata.get('MONGO_CLUSTER')
database = userdata.get('MONGO_DATABAS')

In [6]:
connect_url=f"mongodb+srv://{username}:{password}@{cluster}/?retryWrites=true&w=majority&appName=ProductImageCluster"

In [7]:
client = MongoClient(connect_url, server_api=ServerApi('1'))
db = client[database]
collection = db['products']

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print(f"CLIP loaded on {device}")

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 115MiB/s]


CLIP loaded on cuda


In [9]:
products = list(collection.find({}))
print(f"Processing {len(products)} products...")

Processing 225 products...


In [11]:
from tqdm import tqdm

In [12]:
for i, product in tqdm(enumerate(products),total=len(products)):
    try:
        name = product.get('name', '')
        article = product.get('article_type', '')
        text_desc = f"{name} {article}"
        text_token = clip.tokenize([text_desc], truncate=True).to(device)
        with torch.no_grad():
            text_embedding = model.encode_text(text_token)
            text_embedding = text_embedding.cpu().numpy().flatten()
        norm = torch.linalg.norm(torch.tensor(text_embedding))
        if norm > 0:
            text_embedding = text_embedding / norm.item()
        collection.update_one(
            {'_id': product['_id']},
            {'$set': {'text_embedding': text_embedding.tolist()}}
        )
    except Exception as e:
        print(f"Error for product {product.get('product_id')}: {e}")

100%|██████████| 225/225 [00:52<00:00,  4.29it/s]
